## Web Crawker that collects information on the Hyakumeisan

Information obatain from the excellent web site momonayama.net by Inpossi.

In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup as bs
import string

In [ ]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [ ]:
#Information dowloaded from momonayama.net#
base_path = "https://www.momonayama.net/hundred_mt_individually_data/basic_data/basic_data"
hyakumeisan = []
for i in range(99):
    mount = {}
    raw_html = simple_get(base_path + '-' + str(i+1).zfill(3) + '.html')
    bs_html = bs(raw_html,'html.parser')
    # extract Name
    div = bs_html.findAll("div", {'class':'h2_main_title-01'})
    mount['Nam'] = ''.join(x for x in div[0].h2.text if x not in '\r\n\t')

    # extract Elevation
    div = bs_html.findAll("div", {'class':'bas_pro-01'})
    mount['Lat'] = ''.join(x for x in div[0].p.text if x not in '\r\n\t')

    # extract location
    div = bs_html.findAll("div", {'class':'bas_alt-01'})
    mount['Ele'] = ''.join(x for x in div[0].p.text if x not in '\r\n\t')

    # extract Area
    div = bs_html.findAll("div", {'class':'bas_add-01'})
    mount['Loc'] = ''.join(x for x in div[0].p.text if x not in '\r\n\t')
    print(i+1,mount['Nam'], mount['Ele'])
    hyakumeisan.append(mount)